In [2]:
from typing import Any, Optional, Tuple
import flax.linen as nn
import jax
import jax.numpy as jnp
from flax.core.frozen_dict import FrozenDict, freeze, unfreeze
from flax.linen import combine_masks, make_causal_mask
from flax.linen.attention import dot_product_attention_weights
from flax.traverse_util import flatten_dict, unflatten_dict
from jax import lax
from transformers.modeling_flax_outputs import (
    FlaxBaseModelOutputWithPastAndCrossAttentions,
    FlaxCausalLMOutputWithCrossAttentions,
)
from transformers.modeling_flax_utils import ACT2FN, FlaxPreTrainedModel, append_call_sample_docstring
from transformers.utils import add_start_docstrings, add_start_docstrings_to_model_forward, logging
from transformers.models.gpt2.configuration_gpt2 import GPT2Config

In [3]:
logger = logging.get_logger(__name__)
_CHECKPOINT_FOR_DOC = "openai-community/gpt2"
_CONFIG_FOR_DOC = "GPT2Config"

In [4]:
GPT2_START_DOCSTRING = r"""

    This model inherits from [`FlaxPreTrainedModel`]. Check the superclass documentation for the generic methods the
    library implements for all its model (such as downloading or saving, resizing the input embeddings, pruning heads
    etc.)

    This model is also a Flax Linen
    [flax.nn.Module](https://flax.readthedocs.io/en/latest/_autosummary/flax.nn.module.html) subclass. Use it as a
    regular Flax Module and refer to the Flax documentation for all matter related to general usage and behavior.

    Finally, this model supports inherent JAX features such as:

    - [Just-In-Time (JIT) compilation](https://jax.readthedocs.io/en/latest/jax.html#just-in-time-compilation-jit)
    - [Automatic Differentiation](https://jax.readthedocs.io/en/latest/jax.html#automatic-differentiation)
    - [Vectorization](https://jax.readthedocs.io/en/latest/jax.html#vectorization-vmap)
    - [Parallelization](https://jax.readthedocs.io/en/latest/jax.html#parallelization-pmap)

    Parameters:
        config ([`GPT2Config`]): Model configuration class with all the parameters of the model.
            Initializing with a config file does not load the weights associated with the model, only the
            configuration. Check out the [`~FlaxPreTrainedModel.from_pretrained`] method to load the model weights.
        dtype (`jax.numpy.dtype`, *optional*, defaults to `jax.numpy.float32`):
            The data type of the computation. Can be one of `jax.numpy.float32`, `jax.numpy.float16` (on GPUs) and
            `jax.numpy.bfloat16` (on TPUs).

            This can be used to enable mixed-precision training or half-precision inference on GPUs or TPUs. If
            specified all the computation will be performed with the given `dtype`.

            **Note that this only specifies the dtype of the computation and does not influence the dtype of model
            parameters.**

            If you wish to change the dtype of the model parameters, see [`~FlaxPreTrainedModel.to_fp16`] and
            [`~FlaxPreTrainedModel.to_bf16`].
"""

GPT2_INPUTS_DOCSTRING = r"""
    Args:
        input_ids (`numpy.ndarray` of shape `(batch_size, input_ids_length)`):
            `input_ids_length` = `sequence_length`. Indices of input sequence tokens in the vocabulary.

            Indices can be obtained using [`AutoTokenizer`]. See [`PreTrainedTokenizer.encode`] and
            [`PreTrainedTokenizer.__call__`] for details.

            [What are input IDs?](../glossary#input-ids)
        attention_mask (`numpy.ndarray` of shape `(batch_size, sequence_length)`, *optional*):
            Mask to avoid performing attention on padding token indices. Mask values selected in `[0, 1]`:

            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.

            [What are attention masks?](../glossary#attention-mask)
        position_ids (`numpy.ndarray` of shape `(batch_size, sequence_length)`, *optional*):
            Indices of positions of each input sequence tokens in the position embeddings. Selected in the range `[0,
            config.max_position_embeddings - 1]`.
        past_key_values (`Dict[str, np.ndarray]`, *optional*, returned by `init_cache` or when passing previous `past_key_values`):
            Dictionary of pre-computed hidden-states (key and values in the attention blocks) that can be used for fast
            auto-regressive decoding. Pre-computed key and value hidden-states are of shape *[batch_size, max_length]*.
        output_attentions (`bool`, *optional*):
            Whether or not to return the attentions tensors of all attention layers. See `attentions` under returned
            tensors for more detail.
        output_hidden_states (`bool`, *optional*):
            Whether or not to return the hidden states of all layers. See `hidden_states` under returned tensors for
            more detail.
        return_dict (`bool`, *optional*):
            Whether or not to return a [`~utils.ModelOutput`] instead of a plain tuple.
"""

In [5]:
# Any 是 Python 的类型注解，来自 typing 模块，意思是“可以是任何类型”。
# 它本身不影响运行时行为，只是为了类型检查工具（如 mypy、Pyright）识别。
class FlaxConv1D(nn.Module):
    features: int  # 输出特征数，即卷积核的数量，决定输出通道维度
    use_bias: bool = True  # 是否使用偏置项
    # 表示这个 dtype 参数可以是任何类型（虽然实际使用中你期望它是 jnp.dtype 类型）。
    # 在混合精度训练（如 fp16/bfloat16）中，用于控制是否使用低精度参与前向和反向计算。
    dtype: Any = jnp.float32  # 数据类型，可用于混合精度等优化
    # precision（数值计算精度提示）：
    # 是 jax.lax.dot_general 的一个参数，控制 XLA 编译器在矩阵乘法等操作中的数值精度策略
    # 典型取值（来自 jax.lax.Precision）：
    # None（默认）
    # 'fastest'（追求速度）
    # 'high'（高精度）
    # 'default'（JAX的默认折中）
    # 用于微调 性能 vs 数值稳定性 的权衡，特别是在 TPU 上很重要。
    precision: Any = None  # 控制 XLA 编译时的精度选项，用于优化性能或精度权衡
    # @nn.compact 是 Flax 的装饰器，用于定义模块的“紧凑模式”。
    # 在这个模式中，子层（如 self.param(...), nn.Dense(...)）直接写在 __call__ 中定义。
    # 它的作用是：
    # 允许在 __call__ 方法中创建参数和子模块（否则需要在 setup() 方法中定义）
    # 使代码更简洁直观（常用于小模块，如 MLP 层）
    @nn.compact
    def __call__(self, inputs):
        # 将输入转换为指定dtype，保证计算过程中的数值一致性和稳定性
        inputs = jnp.asarray(inputs, self.dtype)
        # 定义可训练权重 kernel，初始化 shape 为 (features, 输入通道数)
        # 类似于Dense层的权重矩阵，但Flax中习惯将[输出, 输入]的排列
        kernel = self.param("kernel", jax.nn.initializers.normal(stddev=0.02), (self.features, inputs.shape[-1]))
        # 转置 kernel，使其 shape 从 (features, in_dim) → (in_dim, features)
        # 原因是 dot_general 中 inputs 的最后一个维度与 kernel 的第一个维度对齐
        kernel = jnp.asarray(kernel.transpose(), self.dtype)
        # 进行通用张量乘法（dot_general）：
        # 等价于：y = inputs @ kernel，其中 inputs.shape=[..., in_dim]，kernel.shape=[in_dim, features]
        # 配置 (((inputs.ndim - 1,), (0,)), ((), ()))：
        #     inputs 的最后一维 与 kernel 的第 0 维做点积；
        #     其余维度保持不变（批次维或序列长度等）
        y = lax.dot_general(inputs, kernel, (((inputs.ndim - 1,), (0,)), ((), ())), precision=self.precision)
        # 若设置使用偏置，则添加一组 shape 为 (features,) 的偏置项，逐元素加到输出中
        if self.use_bias:
            bias = self.param("bias", jax.nn.initializers.zeros, (self.features,))
            bias = jnp.asarray(bias, self.dtype)
            y = y + bias
        return y

In [6]:
config=GPT2Config()

In [7]:
config.max_position_embeddings

1024

In [8]:
jnp.ones((1,5))

INFO:2025-05-31 23:47:00,832:jax._src.xla_bridge:924: Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:2025-05-31 23:47:00,846:jax._src.xla_bridge:924: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


Array([[1., 1., 1., 1., 1.]], dtype=float32)

In [9]:
make_causal_mask(
                jnp.ones((1,5), dtype="bool"), dtype="bool"
            )

Array([[[[ True, False, False, False, False],
         [ True,  True, False, False, False],
         [ True,  True,  True, False, False],
         [ True,  True,  True,  True, False],
         [ True,  True,  True,  True,  True]]]], dtype=bool)

In [ ]:
# 总结（设计深意）
# ✅ 解决问题：将当前 token 的 KV 信息拼接进缓存，避免重复计算所有历史 KV；
# ✅ 效率优化：使用 lax.dynamic_update_slice 原地更新，避免不必要的张量复制；
# ✅ 跨步状态维护：利用 Flax 的 self.variable() 实现跨 forward 的缓存管理；
# ✅ 解码核心机制：此函数是 autoregressive decoding（如 beam search、sampling）的关键构件；
# ✅ 与 PyTorch 区别：Flax 在模块内部显式管理 cache state，更显函数式设计哲学。

In [10]:
jnp.broadcast_to(
                jnp.arange(2) < 2 + 1,
                tuple([2]) + (1, 1,2),
            )

Array([[[[ True,  True]]],


       [[[ True,  True]]]], dtype=bool)

In [12]:
class FlaxGPT2Attention(nn.Module):
    config: GPT2Config  # 包含 GPT2 的超参数
    dtype: jnp.dtype = jnp.float32   # 控制内部张量的数值类型，支持混合精度
    causal: bool = True # 是否启用因果遮蔽，决定是否是自回归模式（如 decoder）
    is_cross_attention: bool = False # 是否为交叉注意力（decoder 中的 encoder-decoder attention）
    def setup(self):
        config = self.config  
        self.embed_dim = config.hidden_size # 嵌入维度，也是 attention 输入/输出的总通道数
        self.num_heads = config.num_attention_heads # 多头注意力中的“头”数量
        self.head_dim = self.embed_dim // self.num_heads # 每个头的维度，GPT2中一般为64（如 hidden_size=768, num_heads=12）
        # 在交叉注意力中：
        # q_attn 仅对 decoder 输入做 Q 投影；
        # c_attn 对 encoder 的输出做 K 和 V 投影（因此是 2 倍维度）。
        if self.is_cross_attention:
            self.c_attn = FlaxConv1D(2 * self.embed_dim, dtype=self.dtype)
            self.q_attn = FlaxConv1D(self.embed_dim, dtype=self.dtype)
        # 在自注意力中：
        # c_attn 一次性生成 Q、K、V，拼接在一起；
        # 用一个 Conv1D 实现一次性投影（GPT-2 的原始做法，效率优于分别建三层）
        else:
            self.c_attn = FlaxConv1D(3 * self.embed_dim, dtype=self.dtype)
        # 用于注意力输出的线性变换（即 attn_output @ W_o），回到原始维度。
        self.c_proj = FlaxConv1D(self.embed_dim, dtype=self.dtype)
        # residual dropout，用于 transformer 中的残差路径，在训练时随机屏蔽一部分元素，防止过拟合。
        self.resid_dropout = nn.Dropout(rate=config.resid_pdrop)
        # 提前生成 causal mask：
        # 它是一个 [1, 1, seq_len, seq_len] 的布尔张量；
        # 用于保证 decoder 在第 i 个 token 时只关注 ≤ i 的位置；
        # 提前创建是为了避免在前向传播时每次重复计算。
        if self.causal:
            self.causal_mask = make_causal_mask(
                jnp.ones((1, config.max_position_embeddings), dtype="bool"), dtype="bool"
            )
    # 拆分头 -->(b,s,h,dk)
    def _split_heads(self, hidden_states):
        return hidden_states.reshape(hidden_states.shape[:2] + (self.num_heads, self.head_dim))
    # 合并头 -->(b,s,d)
    def _merge_heads(self, hidden_states):
        return hidden_states.reshape(hidden_states.shape[:2] + (self.embed_dim,))
    # 将当前时间步的 key、value 拼接进缓存（cache），以支持自回归解码过程中的高效推理。
    # 使用方式类似于 GPT 或 T5 的 autoregressive decoding。
    # 来自 Flax 官方 attention 实现的适配版本。
    @nn.compact
    def _concatenate_to_cache(self, key, value, query, attention_mask):
        # 缓存变量的创建/读取
        # 判断是否已有缓存（即是否是首次推理）；
        # 解码过程中的第一个 token 会触发变量创建，而后续则读取和更新。
        is_initialized = self.has_variable("cache", "cached_key")
        # cache 是 Flax 中一个专用的可变状态容器；
        # 创建缓存变量：cached_key, cached_value 预留空间；
        # cache_index 记录当前写入位置（类似 指针），主要用于确保序列逐 token 更新。
        # 若变量已存在（is_initialized == True），这几句不会覆盖变量内容，只返回句柄。
        cached_key = self.variable("cache", "cached_key", jnp.zeros, key.shape, key.dtype)
        cached_value = self.variable("cache", "cached_value", jnp.zeros, value.shape, value.dtype)
        cache_index = self.variable("cache", "cache_index", lambda: jnp.array(0, dtype=jnp.int32))
        # 如果缓存已存在，进行增量更新
        if is_initialized:
            # 提取张量维度，支持多批次（batch）；
            # max_length 是缓存的序列最大长度，来自配置；
            # num_heads 和 depth_per_head 用于处理多头注意力。*batch_dims会是元组的样子(2,)
            *batch_dims, max_length, num_heads, depth_per_head = cached_key.value.shape
            # 构造 lax.dynamic_update_slice 所需的索引；
            # 每次只更新当前时间步的 key/value，而非整段替换；
            # indices 指定从哪个位置写入，例如写入第 cur_index 个位置。
            cur_index = cache_index.value
            indices = (0,) * len(batch_dims) + (cur_index, 0, 0) # (0,cur_index, 0, 0)
            # 将当前 key / value 写入缓存；
            # dynamic_update_slice 是 XLA 中用于写入张量特定位置的原语（避免新建张量）；
            # 写入后更新变量值，以便下次使用。
            key = lax.dynamic_update_slice(cached_key.value, key, indices)
            value = lax.dynamic_update_slice(cached_value.value, value, indices)
            cached_key.value = key
            cached_value.value = value
            # 注意 GPT-2/decoder 的 decoding 是按 token 或 token block 来；
            # 所以更新后需要前移 cache_index。
            num_updated_cache_vectors = query.shape[1]
            cache_index.value = cache_index.value + num_updated_cache_vectors
            # 缓存部分的掩码 
            pad_mask = jnp.broadcast_to(
                jnp.arange(max_length) < cur_index + num_updated_cache_vectors,
                tuple(batch_dims) + (1, num_updated_cache_vectors, max_length),
            )
            # 将动态 causal mask 与传入的 attention mask 合并
            # 得到最终的 attention mask（用于注意力得分遮蔽）。
            attention_mask = combine_masks(pad_mask, attention_mask)
        # 返回更新后的状态
        # 输出新的 key/value（含缓存逻辑），以及更新后的 attention mask；
        # 在解码中，这些会被用于计算 QK^T、softmax 等操作。
        return key, value, attention_mask

    def __call__(
        self,
        hidden_states,
        key_value_states: Optional[jnp.ndarray] = None, # 表示来自 encoder 的上下文，用于 decoder 的 cross-attention
        attention_mask=None,
        deterministic: bool = True,
        init_cache: bool = False,
        output_attentions: bool = False,
    ):
        # 若传入了 key_value_states，说明当前是 cross-attention 层
        is_cross_attention = key_value_states is not None
        batch_size = hidden_states.shape[0]
        # QKV 构建（按是否 cross attention）
        if not is_cross_attention:
            qkv_out = self.c_attn(hidden_states)
            query, key, value = jnp.split(qkv_out, 3, axis=2)
        else: # 在 cross-attention 中，query 来自 decoder 输入，key/value 来自 encoder 输出；
            q_out = self.q_attn(hidden_states)
            (query,) = jnp.split(q_out, 1, axis=2)
            kv_out = self.c_attn(key_value_states)
            key, value = jnp.split(kv_out, 2, axis=2)
        # 拆分多头（[batch, seq, dim] → [batch, seq, n_heads, head_dim]）
        query = self._split_heads(query)
        key = self._split_heads(key)
        value = self._split_heads(value)
        # 准备注意力掩码
        query_length, key_length = query.shape[1], key.shape[1] # q_len,k_len
        if self.causal:
            if self.has_variable("cache", "cached_key"):
                mask_shift = self.variables["cache"]["cache_index"] # 当前时间步位置
                # 之前缓存的key序列长度
                max_decoder_length = self.variables["cache"]["cached_key"].shape[1] 
                # 利用 lax.dynamic_slice 动态裁剪出 [query_length, max_length] 的因果遮蔽；
                causal_mask = lax.dynamic_slice(
                    self.causal_mask, (0, 0, mask_shift, 0), (1, 1, query_length, max_decoder_length)
                )
            else: # 没有缓存的情况
                causal_mask = self.causal_mask[:, :, :query_length, :key_length]
            # 在批次轴广播 (b,1,q_len,k_len)
            causal_mask = jnp.broadcast_to(causal_mask, (batch_size,) + causal_mask.shape[1:])

        # attention mask 转为 bias（float mask）
        # 将布尔遮蔽转换为数值遮蔽；
        # masked 的位置被赋值 -inf（即最小 float 值），以使 softmax 输出趋于 0；
        if attention_mask is not None and self.causal:
            attention_mask = jnp.broadcast_to(jnp.expand_dims(attention_mask, axis=(-3, -2)), causal_mask.shape)
            # 合并填充和因果掩码
            attention_mask = combine_masks(attention_mask, causal_mask)
        elif self.causal: # 因果,自回归的情况 没传入attention_mask
            attention_mask = causal_mask
        elif attention_mask is not None: # 传入了attention_mask,编码器的情况
            attention_mask = jnp.expand_dims(attention_mask, axis=(-3, -2))
        dropout_rng = None
        if not deterministic and self.config.attn_pdrop > 0.0:
            dropout_rng = self.make_rng("dropout") # drop 流

        # 若为 decoder 的自回归推理阶段，则用 cache；
        # cache 中存的是历史生成的 key/value；
        # 每步只输入一个 token，但 key/value 会动态增长
        # init_cache是个标记,标记缓存状态是否已经创建,满足条件,就更新状态,attention_mask这时其实是因果掩码
        if self.causal and (self.has_variable("cache", "cached_key") or init_cache):
            key, value, attention_mask = self._concatenate_to_cache(key, value, query, attention_mask)
        # attention mask 转为 bias（float mask）
        # 遮挡位置会是很大的负数
        if attention_mask is not None:
            attention_bias = lax.select(
                attention_mask > 0,
                jnp.full(attention_mask.shape, 0.0).astype(self.dtype),
                jnp.full(attention_mask.shape, jnp.finfo(self.dtype).min).astype(self.dtype),
            )
        else: 
            attention_bias = None
        # 注意力权重计算
        # 核心注意力函数（Flax 封装的 softmax(QK^T + bias)）；
        # 注意这里权重仅 softmax 后的概率，还没乘 V；
        # dropout_rng 仅训练时有效。
        attn_weights = dot_product_attention_weights(
            query,
            key,
            bias=attention_bias,
            dropout_rng=dropout_rng,
            dropout_rate=self.config.attn_pdrop,
            deterministic=deterministic,
            dtype=self.dtype,
            precision=None,
        )
        # attention 输出计算
        # einsum 实现注意力值加权求和；
        # 合并多头；
        # 最后线性映射回残差路径所需维度，添加 dropout；
        attn_output = jnp.einsum("...hqk,...khd->...qhd", attn_weights, value)
        attn_output = self._merge_heads(attn_output)
        attn_output = self.c_proj(attn_output)
        attn_output = self.resid_dropout(attn_output, deterministic=deterministic)
        outputs = (attn_output, attn_weights) if output_attentions else (attn_output,)
        return outputs

In [13]:
class FlaxGPT2MLP(nn.Module):
    config: GPT2Config  # 模型配置
    intermediate_size: int  # 中间层维度
    dtype: jnp.dtype = jnp.float32   # 参数和计算所用的数值精度，默认 float32
    def setup(self):
        embed_dim = self.config.hidden_size # 输入/输出特征维度（GPT中通常为768/1024/1280等）
        # 输入从 hidden_size -> intermediate_size 的线性层（使用Conv1D实现，等价于 Dense）
        self.c_fc = FlaxConv1D(self.intermediate_size, dtype=self.dtype)
        # 激活后的特征再映射回原始维度：intermediate_size -> hidden_size
        self.c_proj = FlaxConv1D(embed_dim, dtype=self.dtype)
        self.act = ACT2FN[self.config.activation_function]  # 激活函数，如 GELU、ReLU 等
        self.dropout = nn.Dropout(rate=self.config.resid_pdrop)  # 残差 dropout，提升泛化能力，控制训练阶段随机性

    def __call__(self, hidden_states, deterministic: bool = True):
        # 第一步：线性变换，将 hidden_states 从 [batch, seq, hidden] 映射为 [batch, seq, intermediate]
        hidden_states = self.c_fc(hidden_states)
        # 第二步：非线性激活，引入复杂特征变换，使模型具有更强表达力（核心提升点）
        hidden_states = self.act(hidden_states)
        # 第三步：投影回原始维度，保持残差连接一致性（原始 residual 需要维度一致）
        hidden_states = self.c_proj(hidden_states)
         # 第四步：Dropout（仅在训练时启用），防止过拟合，改善训练稳定性
        hidden_states = self.dropout(hidden_states, deterministic=deterministic)
        return hidden_states  # 输出：[batch, seq, hidden]

In [ ]:
# causal=False, is_cross_attention=True 使用跨注意力
# causal=True,is_cross_attention=False 使用解码器自注意力

In [15]:
class FlaxGPT2Block(nn.Module):
    config: GPT2Config  # 配置类，包含所有结构超参数
    dtype: jnp.dtype = jnp.float32   # 精度设定，通常为 float32 / bfloat16
    def setup(self):
        hidden_size = self.config.hidden_size
        # inner_dim：MLP 的中间层维度
        inner_dim = self.config.n_inner if self.config.n_inner is not None else 4 * hidden_size
        # 第一个 LayerNorm，用于 Attention 之前（Pre-LN 架构）
        self.ln_1 = nn.LayerNorm(epsilon=self.config.layer_norm_epsilon, dtype=self.dtype)
        # 自注意力模块（支持缓存机制，用于自回归生成）
        self.attn = FlaxGPT2Attention(self.config, dtype=self.dtype)
         # 第二个 LayerNorm，用于 FFN 之前
        self.ln_2 = nn.LayerNorm(epsilon=self.config.layer_norm_epsilon, dtype=self.dtype)
        # 如果开启 cross-attention（如 GPT 解码器对 encoder 输出进行 cross-attend）
        if self.config.add_cross_attention:
            # 创建 cross attention 模块：本质仍是一个 MultiHeadAttention，只是将 encoder 的输出作为 key/value
            # 注意:交叉时,causal=False, is_cross_attention=True 表示不是用因果掩码,是用交叉注意力机制
            self.crossattention = FlaxGPT2Attention(
                config=self.config, dtype=self.dtype, causal=False, is_cross_attention=True
            )
            # 第三个 LayerNorm，cross-attn 使用独立的归一化层
            self.ln_cross_attn = nn.LayerNorm(epsilon=self.config.layer_norm_epsilon, dtype=self.dtype)
        # 前馈网络模块（MLP），包含两层 Conv1D + 激活函数 + Dropout
        self.mlp = FlaxGPT2MLP(self.config, inner_dim, dtype=self.dtype)

    def __call__(
        self,
        hidden_states,
        attention_mask=None,
        encoder_hidden_states: Optional[jnp.ndarray] = None,
        encoder_attention_mask: Optional[jnp.ndarray] = None,
        deterministic: bool = True,  # 控制 dropout 行为（训练时 False，推理时 True）
        init_cache: bool = False,  # 自回归生成时启用，初始化缓存结构
        output_attentions: bool = False,  # 是否返回注意力权重（debug/可视化用）
    ):
         # ====== 自注意力前处理 ======
        residual = hidden_states   # 保存残差连接用的输入
        hidden_states = self.ln_1(hidden_states)  # LayerNorm 在注意力之前
        attn_outputs = self.attn(  # 自注意力机制（多头、带缓存、自因果mask）
            hidden_states,
            attention_mask=attention_mask,
            deterministic=deterministic,
            init_cache=init_cache,
            output_attentions=output_attentions,
        )
        # 注意力输出（不含 residual）
        attn_output = attn_outputs[0]  # output_attn: a, (attentions)
        outputs = attn_outputs[1:]  # 其余是注意力权重（如果请求输出）
         # 残差连接：原始输入 + 注意力输出
        hidden_states = attn_output + residual

         # ====== Cross Attention（可选）======
        if encoder_hidden_states is not None:
            # 需要启用 cross_attention
            if not hasattr(self, "crossattention"):
                raise ValueError(
                    f"If `encoder_hidden_states` are passed, {self} has to be instantiated with "
                    "cross-attention layers by setting `config.add_cross_attention=True`"
                )
            
            residual = hidden_states
            hidden_states = self.ln_cross_attn(hidden_states)
            # cross-attention：将 encoder 的输出作为 key/value，对当前序列 query 做交叉注意力
            cross_attn_outputs = self.crossattention(
                hidden_states,
                key_value_states=encoder_hidden_states,
                attention_mask=encoder_attention_mask,
                deterministic=deterministic,
                output_attentions=output_attentions,
            )
            attn_output = cross_attn_outputs[0]
             # 残差连接
            hidden_states = residual + attn_output
            outputs = outputs + cross_attn_outputs[1:] # 拼接 cross-attn 权重
        # ====== 前馈网络（MLP）=======
        residual = hidden_states
        hidden_states = self.ln_2(hidden_states)  # 第二次 LayerNorm
        # 两层线性层 + 激活函数 + dropout：映射维度→非线性→映射回原始维度
        feed_forward_hidden_states = self.mlp(hidden_states, deterministic=deterministic)
        # 残差连接
        hidden_states = residual + feed_forward_hidden_states
        # ====== 输出结果 ======
        # 第一个为主输出，后续是 attention weights（如果启用）
        outputs = (hidden_states,) + outputs
        return outputs

In [16]:
jnp.arange(5)[None, :]

Array([[0, 1, 2, 3, 4]], dtype=int32)

In [17]:
jnp.broadcast_to(jnp.arange(5)[None, :], (2,5))

Array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]], dtype=int32)

In [18]:
class FlaxGPT2PreTrainedModel(FlaxPreTrainedModel):
    """
    一个抽象类，用于处理权重初始化，并提供一个用于下载和加载预训练模型的简单接口。
    """
    config_class = GPT2Config # 配置
    base_model_prefix = "transformer" # 在权重字典中的基键
    module_class: nn.Module = None # 内部封装的用来干活的主模块
    def __init__(
        self,
        config: GPT2Config,
        input_shape: Tuple = (1, 1),
        seed: int = 0,
        dtype: jnp.dtype = jnp.float32,
        _do_init: bool = True,
        **kwargs,
    ):
        module = self.module_class(config=config, dtype=dtype, **kwargs) # 主模块
        # 调用父类的初始化方法
        super().__init__(config, module, input_shape=input_shape, seed=seed, dtype=dtype, _do_init=_do_init)

    def init_weights(self, rng: jax.random.PRNGKey, input_shape: Tuple, params: FrozenDict = None) -> FrozenDict:
        # 初始化输入张量，模拟模型输入结构（input_ids, attention_mask, position_ids）
        # 注意：只用于初始化权重，不会真实训练或推理
        input_ids = jnp.zeros(input_shape, dtype="i4") # 输入 token id，占位用，全为 0
        attention_mask = jnp.ones_like(input_ids) # 模拟 attention mask，全部为 1 表示无遮挡
        # 广播生成位置编码
        position_ids = jnp.broadcast_to(jnp.arange(jnp.atleast_2d(input_ids).shape[-1]), input_shape)
        params_rng, dropout_rng = jax.random.split(rng)  # 拆分 PRNGKey：分别用于初始化参数和初始化带 dropout 的模块
        # 多流随机源，Flax 通用写法，确保参数/随机过程独立
        rngs = {"params": params_rng, "dropout": dropout_rng}
        # 分支逻辑:if 使用交叉注意力
        if self.config.add_cross_attention:
            # 创建 encoder 侧输入，全为 0，仅用于初始化形状，不参与实际计算
            encoder_hidden_states = jnp.zeros(input_shape + (self.config.n_embd,))
            encoder_attention_mask = attention_mask
            # 初始化模块参数，传入 encoder 输入以激活 cross-attention 路径，确保初始化能覆盖所有子模块
            module_init_outputs = self.module.init(
                rngs,
                input_ids,
                attention_mask,
                position_ids,
                encoder_hidden_states,
                encoder_attention_mask,
                return_dict=False, # 返回元组
            )
        else: # 解码器自注意力初始化
            module_init_outputs = self.module.init(rngs, input_ids, attention_mask, position_ids, return_dict=False)
        # 提取初始化后的权重（FrozenDict），结构对应 nn.Module 中的所有子模块参数
        random_params = module_init_outputs["params"]
        # 如果提供了已有参数，表示是某种 partial restore 模式
        if params is not None:
            # 先对随机初始化的参数处理
            random_params = flatten_dict(unfreeze(random_params))
            params = flatten_dict(unfreeze(params)) # 对传入的参数处理
            for missing_key in self._missing_keys:  # 使用记录的缺失参数 key，从随机初始化结果中填补到已有参数中
                params[missing_key] = random_params[missing_key]
            self._missing_keys = set() # 初始化完成后清除缺失记录，避免下一次重复添加
            return freeze(unflatten_dict(params)) # 返回重新冻结 + 结构还原后的参数树（FrozenDict）
        else: # 如果没传入params,直接返回随机初始化的完整参数树
            return random_params

    def init_cache(self, batch_size, max_length):
         # 创建虚拟输入：用于模拟推理时的输入结构（input_ids, attention_mask, position_ids）
        # 这里只是形状匹配，内容无实际意义，关键在于触发 init_cache 分支
        input_ids = jnp.ones((batch_size, max_length)) # 模拟一批输入，token 全为 1
        attention_mask = jnp.ones_like(input_ids)
        # 构造位置编码，广播成 batch_size x max_length
        position_ids = jnp.broadcast_to(jnp.arange(jnp.atleast_2d(input_ids).shape[-1]), input_ids.shape)
        # init_cache=True 会在子模块中调用forward(call方法) 内部有判断这个标记,之后init_cache module.init调用的是apply方法
        init_variables = self.module.init(
            jax.random.PRNGKey(0), input_ids, attention_mask, position_ids, return_dict=False, init_cache=True
        )
        # 这里返回init_variables中的集合cache
        return unfreeze(init_variables["cache"]) 

    @add_start_docstrings_to_model_forward(GPT2_INPUTS_DOCSTRING)
    def __call__(
        self,
        input_ids,
        attention_mask=None,
        position_ids=None,
        encoder_hidden_states: Optional[jnp.ndarray] = None,
        encoder_attention_mask: Optional[jnp.ndarray] = None,
        params: dict = None,
        past_key_values: dict = None,
        dropout_rng: jax.random.PRNGKey = None,
        train: bool = False,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.return_dict
        # 如果传入了编码器输出 并且没有传入编码器填充掩码
        if encoder_hidden_states is not None and encoder_attention_mask is None:
            batch_size, sequence_length = encoder_hidden_states.shape[:2] # 编码器 b,s
            encoder_attention_mask = jnp.ones((batch_size, sequence_length)) # 默认的编码器填充掩码
        batch_size, sequence_length = input_ids.shape # 目标序列 b,s
        # 如果没传入位置ids
        if position_ids is None:
            # 有缓存的话,必须传入位置ids
            if past_key_values is not None: 
                raise ValueError("Make sure to provide `position_ids` when passing `past_key_values`.")
            # 这个是没缓存的情况 
            position_ids = jnp.broadcast_to(jnp.arange(sequence_length)[None, :], (batch_size, sequence_length))
        # 设置默认的填充掩码
        if attention_mask is None:
            attention_mask = jnp.ones((batch_size, sequence_length))
        # Handle any PRNG if needed
        rngs = {} # 设置默认的rngs dropout流
        if dropout_rng is not None:
            rngs["dropout"] = dropout_rng
        inputs = {"params": params or self.params}
        # 如果传入了 past_key_values，说明缓存（cache）已经初始化；
        # 此时必须向下传递一个私有标志 init_cache，以确保使用缓存；
        # 还必须确保缓存被标记为 mutable（可变），这样它才能在 FlaxGPT2Attention 模块中被修改
        if past_key_values: 
            inputs["cache"] = past_key_values 
            mutable = ["cache"] # 指定可以修改的集合
        else: # 没有缓存的情况
            mutable = False
        outputs = self.module.apply(
            inputs, # 变量字典
            jnp.array(input_ids, dtype="i4"), # "i4" 等价于 jnp.int32，表示 32位有符号整数
            jnp.array(attention_mask, dtype="i4"),
            jnp.array(position_ids, dtype="i4"),
            encoder_hidden_states,
            encoder_attention_mask,
            not train,  # deterministic
            False, # init_cache
            output_attentions,
            output_hidden_states,
            return_dict,
            rngs=rngs,
            mutable=mutable,
        )

        # 将更新的缓存添加到模型输出
        if past_key_values is not None and return_dict:
            outputs, past_key_values = outputs
            outputs["past_key_values"] = unfreeze(past_key_values["cache"])
            return outputs
        # 如果传入了缓存,指定返回元组
        elif past_key_values is not None and not return_dict:
            outputs, past_key_values = outputs
            outputs = outputs[:1] + (unfreeze(past_key_values["cache"]),) + outputs[1:]
        return outputs

In [19]:
class FlaxGPT2BlockCollection(nn.Module):
    config: GPT2Config # 配置
    dtype: jnp.dtype = jnp.float32 # 矩阵运算的数据类型
    def setup(self): # 子模块初始化于此
        # 层集合
        self.blocks = [
            FlaxGPT2Block(self.config, name=str(i), dtype=self.dtype) for i in range(self.config.num_hidden_layers)
        ]

    def __call__(
        self,
        hidden_states,
        attention_mask=None,
        encoder_hidden_states: Optional[jnp.ndarray] = None,
        encoder_attention_mask: Optional[jnp.ndarray] = None,
        deterministic: bool = True,
        init_cache: bool = False,
        output_attentions: bool = False,
        output_hidden_states: bool = False,
        return_dict: bool = True,
    ):
        all_attentions = () if output_attentions else None
        all_hidden_states = () if output_hidden_states else None
        all_cross_attentions = () if (output_attentions and encoder_hidden_states is not None) else None
        # 遍历解码器的每一层
        for block in self.blocks:
            if output_hidden_states: # 添加每个层的输入 carry
                all_hidden_states += (hidden_states,)
            layer_outputs = block(
                hidden_states,
                attention_mask,
                encoder_hidden_states=encoder_hidden_states,
                encoder_attention_mask=encoder_attention_mask,
                deterministic=deterministic,
                init_cache=init_cache,
                output_attentions=output_attentions,
            )
            hidden_states = layer_outputs[0]
            if output_attentions:
                all_attentions += (layer_outputs[1],)
                if encoder_hidden_states is not None: # 编码器输出有传入,说明是交叉注意力
                    all_cross_attentions += (layer_outputs[2],) # 交叉注意力权重
        # 这包含可能的 `None` 值 - `FlaxGPT2Module` 会将它们过滤掉
        outputs = (hidden_states, all_hidden_states, all_attentions, all_cross_attentions)
        return outputs

In [21]:
class FlaxGPT2Module(nn.Module):
    config: GPT2Config 
    dtype: jnp.dtype = jnp.float32

    def setup(self): # 定义的一些子模块
        self.embed_dim = self.config.hidden_size # 嵌入维度
        self.wte = nn.Embed( # 词嵌入
            self.config.vocab_size,
            self.embed_dim,
            embedding_init=jax.nn.initializers.normal(stddev=self.config.initializer_range),
            dtype=self.dtype,
        )
        self.wpe = nn.Embed( # 位置嵌入
            self.config.max_position_embeddings,
            self.embed_dim,
            embedding_init=jax.nn.initializers.normal(stddev=self.config.initializer_range),
            dtype=self.dtype,
        )
        self.dropout = nn.Dropout(rate=self.config.embd_pdrop) # dropout
        self.h = FlaxGPT2BlockCollection(self.config, dtype=self.dtype) # 主模块
        self.ln_f = nn.LayerNorm(epsilon=self.config.layer_norm_epsilon, dtype=self.dtype)
    def __call__(
        self,
        input_ids,
        attention_mask,
        position_ids,
        encoder_hidden_states: Optional[jnp.ndarray] = None, # 编码器输出
        encoder_attention_mask: Optional[jnp.ndarray] = None,
        deterministic=True,
        init_cache: bool = False,
        output_attentions: bool = False,
        output_hidden_states: bool = False,
        return_dict: bool = True,
    ):
        input_embeds = self.wte(input_ids.astype("i4")) 
        position_embeds = self.wpe(position_ids.astype("i4"))
        hidden_states = input_embeds + position_embeds # 隐藏状态:序列中token的表示
        hidden_states = self.dropout(hidden_states, deterministic=deterministic) 
        outputs = self.h(
            hidden_states,
            attention_mask,
            encoder_hidden_states,
            encoder_attention_mask,
            deterministic=deterministic,
            init_cache=init_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0] # transformer decoder的输出
        hidden_states = self.ln_f(hidden_states) # norm
        if output_hidden_states:
            all_hidden_states = outputs[1] + (hidden_states,)
            outputs = (hidden_states, all_hidden_states) + outputs[2:]
        else:
            outputs = (hidden_states,) + outputs[1:]
        if not return_dict:
            return tuple(v for v in outputs if v is not None)
        return FlaxBaseModelOutputWithPastAndCrossAttentions(
            last_hidden_state=hidden_states, # 最后一层的隐藏状态
            hidden_states=outputs[1], 
            attentions=outputs[2],
            cross_attentions=outputs[3],
        )

In [22]:
# FlaxGPT2Model 是面向用户的高层接口（包装器），FlaxGPT2Module 是实际的网络结构。
# 通过 module_class 将两者解耦，是 HuggingFace 在 Flax 中对模块化、灵活性的一种工程化实现。
# FlaxGPT2Model 并不硬编码模块实现，而是通过类变量 module_class 持有结构定义。
# 允许在上层模型复用逻辑的同时，替换底层架构（如换成 FlaxGPTNeoXModule、FlaxGPT2WithCrossAttnModule 等）。
@add_start_docstrings(
    "The bare GPT2 Model transformer outputting raw hidden-states without any specific head on top.",
    GPT2_START_DOCSTRING,
)
class FlaxGPT2Model(FlaxGPT2PreTrainedModel):
    module_class = FlaxGPT2Module

In [24]:
_CHECKPOINT_FOR_DOC

'openai-community/gpt2'

In [25]:
# 这行代码的目的是：向 FlaxGPT2Model.__call__ 方法自动追加 docstring 示例
append_call_sample_docstring(
    FlaxGPT2Model, # 目标类对象，其 __call__ 方法会被补充文档。
    _CHECKPOINT_FOR_DOC, # 一个字符串，代表参考的预训练模型（如 "gpt2"），用于生成示例
    FlaxBaseModelOutputWithPastAndCrossAttentions, # 模型输出的类型，告知用户 .call() 返回的是这个结构
    _CONFIG_FOR_DOC, # 指定配置类（如 "GPT2Config"），用于文档中展示如何加载对应模型
)

In [26]:
class FlaxGPT2LMHeadModule(nn.Module):
    config: GPT2Config
    dtype: jnp.dtype = jnp.float32
    def setup(self): # 定义子模块
        self.transformer = FlaxGPT2Module(self.config, dtype=self.dtype) 
        self.lm_head = nn.Dense(
            self.config.vocab_size,
            use_bias=False,
            dtype=self.dtype,
            kernel_init=jax.nn.initializers.normal(stddev=self.config.initializer_range),
        )

    def __call__(
        self,
        input_ids,
        attention_mask,
        position_ids,
        encoder_hidden_states: Optional[jnp.ndarray] = None,
        encoder_attention_mask: Optional[jnp.ndarray] = None,
        deterministic: bool = True,
        init_cache: bool = False,
        output_attentions: bool = False,
        output_hidden_states: bool = False,
        return_dict: bool = True,
    ):
        outputs = self.transformer(
            input_ids,
            attention_mask,
            position_ids,
            encoder_hidden_states,
            encoder_attention_mask,
            deterministic=deterministic,
            init_cache=init_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        hidden_states = outputs[0] # 经过各个解码器层的输出
        # 如果配置中指定词嵌入共享
        if self.config.tie_word_embeddings:
            shared_kernel = self.transformer.variables["params"]["wte"]["embedding"].T # 转置输入的词嵌入
            # self.lm_head 是一个 nn.Module 的实例（如 nn.Dense）
            # 它定义了权重结构（如 kernel、bias）和前向传播逻辑（__call__），但本身是无状态的。
            # Flax 设计为纯函数式，所以使用 .apply() 显式提供参数。hidden_states 是输入张量
            lm_logits = self.lm_head.apply({"params": {"kernel": shared_kernel}}, hidden_states)
        else: # 如果不共享
            lm_logits = self.lm_head(hidden_states)
        if not return_dict:
            return (lm_logits,) + outputs[1:]
        return FlaxCausalLMOutputWithCrossAttentions(
            logits=lm_logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
            cross_attentions=outputs.cross_attentions,
        )

In [28]:
jnp.ones((1,5), dtype="i4").cumsum(axis=-1) - 1

Array([[0, 1, 2, 3, 4]], dtype=int32)

In [36]:
@add_start_docstrings(
    """
    The GPT2 Model transformer with a language modeling head on top (linear layer with weights tied to the input
    embeddings).
    """,
    GPT2_START_DOCSTRING,
)
class FlaxGPT2LMHeadModel(FlaxGPT2PreTrainedModel):  # GPT2 带语言建模头的完整模型
    module_class = FlaxGPT2LMHeadModule  # 指定用于初始化的核心模块类（定义 forward 行为）
    # 准备生成任务所需的输入结构，尤其适配 cache 的初始设定
    def prepare_inputs_for_generation(self, input_ids, max_length, attention_mask: Optional[jax.Array] = None):
        batch_size, seq_length = input_ids.shape   # 当前输入的批大小与序列长度
        # 初始化用于自回归生成的缓存结构（含 key/value attention 缓存）
        # 注意 cache 长度 = max_length（目标生成长度），而非当前输入长度
        past_key_values = self.init_cache(batch_size, max_length) 
        # GPT2 使用 causal mask，因此可以用静态全 1 的 attention_mask：
        # 即便后续 token 是 padding，仍由 causal mask 自动屏蔽，不影响因果逻辑
        # 使用静态 mask 避免 XLA 编译过程中因 shape 动态变化导致的重编译开销
        extended_attention_mask = jnp.ones((batch_size, max_length), dtype="i4")
        # 如果传入了attention_mask
        if attention_mask is not None:
            # 为了保证位置编码连续性，需要根据 attention_mask 生成 position_ids
            # 用累加方式将有效 token 的位置编码设置为 0,1,2,...（padding 仍为负值）
            position_ids = attention_mask.cumsum(axis=-1) - 1 #  位置ids
            # 更新 extended_attention_mask 的前部，使得有效输入位置保留原 mask
            # 后部保持为 1，因 causal mask 已处理未来 token，无需显式屏蔽
            # operand：要被更新的原始数组。update：包含要写入到 operand 中的新值的数组。
            # start_indices：表示每个维度起始更新位置的标量索引列表
            # 返回一个数组，其对应位置被 update 数组内容替换（插入），其余保持 operand 原值。
            extended_attention_mask = lax.dynamic_update_slice(
                extended_attention_mask, attention_mask.astype("i4"), (0, 0)
            )
        else:  # 若未传入 attention_mask，则默认按顺序构造位置编码
            position_ids = jnp.broadcast_to(jnp.arange(seq_length, dtype="i4")[None, :], (batch_size, seq_length))

        return {
            "past_key_values": past_key_values,  # 初始化的 缓存（key/value）
            "attention_mask": extended_attention_mask,  # 静态或更新后的注意力掩码
            "position_ids": position_ids,    # 当前输入 token 的位置编码
        }
    # 在每轮生成后更新输入状态，用于下一步生成
    def update_inputs_for_generation(self, model_outputs, model_kwargs):
        # 将上一轮输出的缓存继续传入下一轮，用于加速自回归 attention
        model_kwargs["past_key_values"] = model_outputs.past_key_values
        # 自增 position_ids，仅保留最后一个 token 的位置编码，加速推理
        model_kwargs["position_ids"] = model_kwargs["position_ids"][:, -1:] + 1
        return model_kwargs
        
append_call_sample_docstring(
    FlaxGPT2LMHeadModel,
    _CHECKPOINT_FOR_DOC,
    FlaxCausalLMOutputWithCrossAttentions,
    _CONFIG_FOR_DOC,
)

In [37]:
x = jnp.zeros(6)        # 原始数组：6 个 0
y = jnp.ones(3)         # 要更新进去的数组：3 个 1
lax.dynamic_update_slice(x, y, (2,))

Array([0., 0., 1., 1., 1., 0.], dtype=float32)

In [33]:
# 若更新数组过大超出原数组尾部，会自动向前调整插入位置以适配：
lax.dynamic_update_slice(x, y, (3,))

Array([0., 0., 0., 1., 1., 1.], dtype=float32)

In [34]:
lax.dynamic_update_slice(x, y, (5,))

Array([0., 0., 0., 1., 1., 1.], dtype=float32)

In [35]:
# 二维数组更新示例：
x = jnp.zeros((4, 4))       # 4x4 零矩阵
y = jnp.ones((2, 2))        # 2x2 的更新值矩阵
lax.dynamic_update_slice(x, y, (1, 2))

Array([[0., 0., 0., 0.],
       [0., 0., 1., 1.],
       [0., 0., 1., 1.],
       [0., 0., 0., 0.]], dtype=float32)

In [31]:
__all__ = ["FlaxGPT2LMHeadModel", "FlaxGPT2Model", "FlaxGPT2PreTrainedModel"]